# Storage of predicates and querying

Storing predicates as binary and joint entities as id

relationship table
rel - Xid - Yid - True

id table
X - id (autogenerated)
Y - id
('temperature','numeric continuous') - id

exists - Xid - NA - True
role  - Xid - set - True
role - Xid - instance - True
role - Xid - attribute - True
is_attribute_of - Xid- Yid - True





In [163]:
import pandas as pd
conceptlist = ['abandonment', 'abandonment depression', 'abuse', 'awareness', 'client', 'contempt', 'decade', 'depression', 'emotional flashback', 'fear', 'feeling', 'flashback', 'home', 'matter', 'mother', 'parent', 'safety', 'sensation', 'shame', 'someone', 'toxic shame']


## 1. Exists
exists pandas series - index is conceptlist, value is True, False
0. initially autopopulated as True
1. storage function = exists(X), does_not_exist(X)
2. querying function = does_X_exist(X) - returns true or false - will be used for cues, deduction
3. contradiction function = can_X_exist(X), can_X_not_exist(X)
4. meta function = source(predicate,sent)

5. deductions: if exists, then intiate roleslist

In [21]:
existence = pd.Series(['True']*len(conceptlist))
existence.index = conceptlist
existence.head(5)

def store_exists(X):
    global existence
    if can_X_exist(X):
        existence[X] = 'True'
        return True
    else:
        print('Cannot store X as exists')
        return False
    

def store_does_not_exist(X):
    global existence
    existence[X] = 'False'

def query_does_X_exist(X):
    global existence
    if X in existence:
        return existence[X]
    else:
        return "Possible"

def can_X_exist(X):
    global existence
    if query_does_X_exist(X)=='False':
        return False
    else:
        return True
        
# contradiction function pending


## 2. Roles
-stores the role for X;
-pandas series: multi_index (X,role) = 'True','False','Possible';
-initiate - possible for all exists(X) - this will be in deduction table;
-conceptlist,all roles = Possible

storage functions:
1. store_plays_role(X,role)
2. store_does_not_play_role(X,role)

query function:
1. query_does_X_play_role_Y(X,Y)
2. query_what_roles_does_X_play(X,Y)
3. query_what_plays_role_Y(Y)


contradiction functions:
1. can_X_play_role_Y(X,Y) - checks if x exists, y is a valid role from rolelist, checks if not stored role as False. for set and instances additional caveat - during storage, stores role as set, then instance = False and viceversa


In [69]:
roleslist = ['set','instance','phenomenon','system','component','attribute','unit','tool','application','transformer']
roles = pd.Series(['Possible']*len(conceptlist)*len(roleslist),
                  [(c,r) for c in conceptlist for r in roleslist])


In [71]:
def store_role(X,role):
    global roles
    if can_X_play_role_Y:
        roles[(X,role)]='True'
        return True
    else:
        print(X,'cannot play role',Y)
        return False

def store_does_not_play_role(X,role):
    global roles
    if Y in roleslist:
        roles[(X,role)]='False'

def can_X_play_role_Y(X,Y):
    if Y in roleslist:
        if roles[(X,Y)]=='False':
            return False
        else:
            return True
    else:
        return False
    
def query_does_X_play_role_Y(X,Y):
    if (Y in roleslist) & (X in conceptlist):
        return roles[(X,Y)]
    else:
        return 'False'

def query_what_roles_does_X_play(X):
    return list(roles[X])

def query_what_plays_role_Y(Y):
    if Y in roleslist:
        return [c for (c,r) in roles.index if roles[(c,r)]!='True']

## 3. Also known as (synonyms)

- aka series: pairs of concepts
- not initiating every possible combination as possible . 
storage functions
1. store_is_synonym(Xnew,Xorig)
2. store_is_not_synonym(X,Xorig)

query functions
1. query_is_synonym(X,Y)
2. query_what_is_X_aka(X)

contradiction functions
1. can_synonym_X_Y(X,Y)

treat it like 'identical to' but for instances only. and use categorical deduction. 

In [164]:
aka = pd.Series(data = ['True']*len(conceptlist),index = [frozenset([i]) for i in conceptlist])

# every concept should occur in only one of the sets.so if it occurs in different sets, it is not a synonym

def store_is_synonym(X,Y):
    # if X is a synonym of Z, Z is automatically made a synonym of X
    global aka
    foundX = 0
    foundY = 0
    for ind in aka.index:
        if X in ind:
            indX = ind
            foundX = 1
        if Y in ind:
            indY = ind
            foundY = 1
    if foundX==0:
        indX = frozenset([X])
    if foundY==0:
        indY= frozenset([Y])
    indnew = indX.union(indY)
    if foundX ==1:
        aka = aka.drop(labels = [indX])
    if (foundY==1) & (indY!=indX):
        aka = aka.drop(labels = [indY])
    aka = aka.append(pd.Series(data = ['True'],index= [indnew]))
    
        
        

def store_is_not_synonym(Xnew,Xorig):
    # if Xnew is not a synonym of Xorig, and was previously in the same set, only Xnew is removed and rest of set is retained
    global aka
    foundXnew = 0
    foundXorig = 0
    for ind in aka.index:
        if (Xnew in ind) & (Xorig in ind):
            indXnew = ind
            foundXnew = 1
        if Xorig in ind:
            indXorig = ind
            foundXorig = 1
    if foundX==0:
        indXnew = frozenset([X])
    if foundY==0:
        indXorig= frozenset([Y])
    indnew = indX.union(indY)
    if foundX ==1:
        aka = aka.drop(labels = [indX])
    if (foundY==1) & (indY!=indX):
        aka = aka.drop(labels = [indY])
    aka = aka.append(pd.Series(data = ['True'],index= [indnew]))
    
    
def query_is_synonym(X,Y):
    found = 0
    for ind in aka.index:
        if (X in ind) & (Y in ind):
            found = 1
            if aka[ind]=='True':
                return 'True'
            elif aka[ind]=='False':
                return 'False'
            else:
                return 'Possible'
    if found==0:
        return 'Possible'

def query_what_is_X_aka(X):
    found = 0
    for ind in aka.index:
        if X in ind:
            found = 1
            return ind-set([X])
    if found==0:
        return set()
        
def can_X_be_aka_Y(X,Y):
    if (role(X,'instance')!='False') & (role(Y,'instance')!='False'):
        return True
    else:
        return False

IndentationError: expected an indented block (<ipython-input-164-92d64b9bd48d>, line 36)

In [165]:
aka
# store_is_synonym('period','fullstop')
# print(aka)
# store_is_synonym('dot','fullstop')
# print(aka)
# store_is_synonym('period','fullstop')
# print(aka)

(period, fullstop, dot)    True
dtype: object

In [166]:
a = frozenset([1,2])
b = frozenset([2,1])
a==b

True

## 4. Attributes

if X is an attribute, and Y is a set or instance, then X can be an attribute of Y

